In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 151kB/s  eta 0:00:01   |██▍                             | 16.3MB 5.7MB/s eta 0:00:36MB/s eta 0:00:24 6.8MB/s eta 0:00:18     |██████████████████              | 122.9MB 6.8MB/s eta 0:00:14     |████████████████████████▏       | 164.4MB 17.1MB/s eta 0:00:04
     |████████████████████████████████| 204kB 37.7MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')


In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

In [16]:
def minTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT MIN(temperature) as mintemp from washing").first().mintemp

In [18]:
def meanTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT avg(temperature) as meantemp from washing").first().meantemp

In [7]:
def maxTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT MAX(temperature) as maxtemp from washing").first().maxtemp

In [21]:
def sdTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT stddev_pop(temperature) as sdtemp from washing").first().sdtemp

In [26]:
def skewTemperature():    
    return spark.sql("""
SELECT 
    (
    SUM (
        POWER(temperature-%s,3))/(POWER(%s,3)*count(temperature))
    )

as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature

In [29]:
def kurtosisTemperature():    
        return spark.sql("""
SELECT 
    (
    SUM (
        POWER(temperature-%s,4))/(POWER(%s,4)*count(temperature))
    )
as ktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().ktemperature

In [38]:
def correlationTemperatureHardness():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT corr(temperature,hardness) as temperaturehardness from washing").first().temperaturehardness

In [13]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

--2020-09-13 16:14:25--  https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/IBM/skillsnetwork/blob/master/coursera_ds/washing.parquet?raw=true [following]
--2020-09-13 16:14:25--  https://github.com/IBM/skillsnetwork/blob/master/coursera_ds/washing.parquet?raw=true
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/skillsnetwork/raw/master/coursera_ds/washing.parquet [following]
--2020-09-13 16:14:25--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ds/washing.parquet
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ds/washing.parquet [fo

In [36]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
#print(df.count())
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

In [15]:
min_temperature = 0
mean_temperature = 0
max_temperature = 0
sd_temperature = 0
skew_temperature = 0
kurtosis_temperature = 0
correlation_temperature = 0

In [17]:
min_temperature = minTemperature()
print(min_temperature)

80


In [19]:
mean_temperature = meanTemperature()
print(mean_temperature)

90.03800298062593


In [20]:
max_temperature = maxTemperature()
print(max_temperature)

100


In [22]:
sd_temperature = sdTemperature()
print(sd_temperature)

6.098487624200337


In [32]:
skew_temperature = skewTemperature()
print(skew_temperature)
#print(spark.sql("SELECT skewness(temperature) as skvtemperature from washing").first().skvtemperature)

0.010410008042945763


In [37]:
kurtosis_temperature = kurtosisTemperature()
print(kurtosis_temperature)
#print(spark.sql("SELECT kurtosis(temperature) as kurtemp from washing").first().kurtemp)

1.7760730694213112


In [39]:
correlation_temperature = correlationTemperatureHardness()
print(correlation_temperature)

0.017754069047296324


In [40]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2020-09-13 17:01:15--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,540       --.-K/s   in 0s      

2020-09-13 17:01:16 (19.4 MB/s) - ‘rklib.py’ saved [2540/2540]



In [42]:
from rklib import submitAll
import json

key = "Suy4biHNEeimFQ479R3GjA"
email = "101guptaji@gmail.com"
token = "CKege1irXjZ4OaM5"

In [43]:
parts_data = {}
parts_data["FWMEL"] = json.dumps(min_temperature)
parts_data["3n3TK"] = json.dumps(mean_temperature)
parts_data["KD3By"] = json.dumps(max_temperature)
parts_data["06Zie"] = json.dumps(sd_temperature)
parts_data["Qc8bI"] = json.dumps(skew_temperature)
parts_data["LoqQi"] = json.dumps(kurtosis_temperature)
parts_data["ehNGV"] = json.dumps(correlation_temperature)



submitAll(email, token, key, parts_data)


Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"TzU1P","id":"sUpST4RAEeawAApvKZgcCQ~TzU1P~8mMoMPXiEeqxoBJMmIBYfQ","courseId":"sUpST4RAEeawAApvKZgcCQ"}],"paging":{},"linked":{}}
-------------------------
